In [1]:
import cv2
import numpy as np
import random

In [2]:
CV_raw = cv2.imread("./rawdata/CVISGOOD.png", 0)
DIP_raw = cv2.imread("./rawdata/DIPISGOOD.png", 0)

In [3]:
DIP_raw[DIP_raw > 127.5] = 255
DIP_raw[DIP_raw <= 127.5] = 0
CV_raw[CV_raw > 127.5] = 255
CV_raw[CV_raw <= 127.5] = 0

In [4]:
def visual_cryptography1x2(img, n = 2):
    C0 = []
    C1 = []
    for i in range(n):
        for j in range(n):
            pattern = np.zeros((n, n))
            if(i == j):
                pattern[:,i] = 1
                C0.append(pattern)                
            elif(i != j):
                pattern[0,i] = 1
                pattern[1,j] = 1
                C1.append(pattern)

    share1 = np.zeros((img.shape[0], img.shape[1]*n))
    share2 = np.zeros((img.shape[0], img.shape[1]*n))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            x = random.randint(0, 1)
            if img[i,j] == 0: #若原始pixel為黑色
                target = C1[x]
            elif img[i,j] == 255: #若原始pixel為白色
                target = C0[x]
            share1[i, j*n:j*n+n] = target[0,:]
            share2[i, j*n:j*n+n] = target[1,:]
    result = np.logical_or(share1.astype(bool), share2.astype(bool)).astype(int)
    result[result==0]=255
    result[result==1]=0

    shares = []
    for share in [share1,share2]:
        share[share==0]=255
        share[share==1]=0
        shares.append(share)
    return shares, result

In [5]:
shares, result = visual_cryptography1x2(DIP_raw)
cv2.imwrite("./output/DIP1x2_share1+share2.png", result)
cv2.imwrite("./output/DIP1x2_share1.png", shares[0])
cv2.imwrite("./output/DIP1x2_share2.png", shares[1])

True

In [6]:
def visual_cryptography1x3(img, n = 3):
    C0 = []
    C1 = []
    for i in range(n):
        for j in range(n):
            for k in range(n):
                pattern = np.zeros((n, n))
                if(i == j and j == k):
                    pattern[:,i] = 1
                    C0.append(pattern)                
                elif(i != j and j != k and i != k):
                    pattern[0,i] = 1
                    pattern[1,j] = 1
                    pattern[2,k] = 1
                    C1.append(pattern)

    share1 = np.zeros((img.shape[0], img.shape[1]*n))
    share2 = np.zeros((img.shape[0], img.shape[1]*n))
    share3 = np.zeros((img.shape[0], img.shape[1]*n))
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            x = random.randint(0, 5)
            y = random.randint(0, 2)
            if img[i,j] == 0: #若原始pixel為黑色
                target = C1[x]
            elif img[i,j] == 255: #若原始pixel為白色
                target = C0[y]
            share1[i, j*n:j*n+n] = target[0,:]
            share2[i, j*n:j*n+n] = target[1,:]
            share3[i, j*n:j*n+n] = target[2,:]
    results = []
    for pair in [[share1,share2],[share1,share3],[share2,share3]]:
        result = np.logical_or(pair[0].astype(bool), pair[1].astype(bool)).astype(int)
        result[result==0]=255
        result[result==1]=0
        results.append(result)
    shares = []
    for share in [share1,share2,share3]:
        share[share==0]=255
        share[share==1]=0
        shares.append(share)
    return shares, results

In [7]:
shares, results = visual_cryptography1x3(DIP_raw)
cv2.imwrite("./output/DIP1x3_share1+share2.png", results[0])
cv2.imwrite("./output/DIP1x3_share1+share3.png", results[1])
cv2.imwrite("./output/DIP1x3_share2+share3.png", results[2])
cv2.imwrite("./output/DIP1x3_share1.png", shares[0])
cv2.imwrite("./output/DIP1x3_share2.png", shares[1])
cv2.imwrite("./output/DIP1x3_share3.png", shares[2])

True

In [8]:
def cheating(img, shares, n=3):
    share1 = shares[0]
    share2 = shares[1]
    share3 = shares[2]
    fake_share1 = np.zeros(share1.shape)
    fake_share2 = np.zeros(share2.shape)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            a = np.where(share1[i, j*n:j*n+n] == 0)[0][0]
            b = np.where(share2[i, j*n:j*n+n] == 0)[0][0]
            filled1 = np.zeros((1, 3))
            filled2 = np.zeros((1, 3))
            c = [0,1,2]
            if(a == b):
                if(img[i,j]==255): #白(Case1)
                    filled1[0,a] = 1
                    filled2[0,a] = 1
                elif(img[i,j]==0): #黑(Case2)
                    c.remove(a)
                    filled1[0,c[0]] = 1
                    filled2[0,c[1]] = 1                    
            elif(a != b):
                if(img[i,j]==255): #白(Case3)
                    c.remove(a)
                    c.remove(b)
                    filled1[0,c[0]] = 1
                    filled2[0,c[0]] = 1                      
                elif(img[i,j]==0): #黑(Case4)
                    filled1[0,a] = 1
                    filled2[0,b] = 1
            fake_share1[i, j*n:j*n+n] = filled1
            fake_share2[i, j*n:j*n+n] = filled2
#             if(random.randint(0, 500)==500):
#                 print(share1[i, j*n:j*n+n])
#                 print(share2[i, j*n:j*n+n])
#                 print(img[i,j])
#                 print(filled1)
#                 print(filled2)
#                 print("--------------------")
    share3[share3==0] = 1
    share3[share3==255] = 0
    results = []
    for fake_share in [fake_share1, fake_share2]:
        result = np.logical_or(fake_share.astype(bool), share3.astype(bool)).astype(int)
        result[result==0]=255
        result[result==1]=0
        results.append(result)
    fake_shares = []
    for fake_share in [fake_share1, fake_share2]:
        fake_share[fake_share==0]=255
        fake_share[fake_share==1]=0
        fake_shares.append(fake_share)        
    return fake_shares, results

In [9]:
fake_shares, results = cheating(CV_raw, shares)
cv2.imwrite("./output/fake_share1.png", fake_shares[0])
cv2.imwrite("./output/fake_share2.png", fake_shares[1])
cv2.imwrite("./output/fake_share1+share3.png", results[0])
cv2.imwrite("./output/fake_share2+share3.png", results[1])

True